# Step 0: Importing Necessary Libraries, Setting Random Seed and Timestamp for Plot titles

## Import Necessary Libraries

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from causalml.inference.meta import BaseTLearner
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import yfinance as yf


## Set random seed for reproducibility


In [45]:
np.random.seed(369)

## Timestamp for plot titles and separate print


In [99]:
timestamp = "Sunday, May 18, 2025" # (For documentation purposes)
print(f"Analysis Run At: {timestamp}")

Analysis Run At: Sunday, May 18, 2025


# Step 1: Fetch real-world stock market data
## Define companies: Tesla (treated), Microsoft (control)

In [100]:
# Data from Jan 2022 to May 2025

treated_ticker = 'TSLA'  # Tesla
control_ticker = 'MSFT'  # Microsoft
vix_ticker = '^VIX'      # VIX index (market volatility)
tnx_ticker = '^TNX'      # 10-year Treasury yield
sp500_ticker = '^GSPC'   # S&P 500 index


## Fetch data from Jan 2022 to May 2025


In [101]:

start_date = '2022-01-01'
end_date = '2025-05-15' 
treated_data = yf.download(treated_ticker, start=start_date, end=end_date)
control_data = yf.download(control_ticker, start=start_date, end=end_date)
vix_data = yf.download(vix_ticker, start=start_date, end=end_date)
tnx_data = yf.download(tnx_ticker, start=start_date, end=end_date)
sp500_data = yf.download(sp500_ticker, start=start_date, end=end_date)

print("Treated Data:" , treated_data)
print("Control Data:" , control_data)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Treated Data: Price            Close        High         Low        Open     Volume
Ticker            TSLA        TSLA        TSLA        TSLA       TSLA
Date                                                                 
2022-01-03  399.926666  400.356659  378.679993  382.583344  103931400
2022-01-04  383.196655  402.666656  374.350006  396.516663  100248300
2022-01-05  362.706665  390.113342  360.336670  382.216675   80119800
2022-01-06  354.899994  362.666656  340.166656  359.000000   90336600
2022-01-07  342.320007  360.309998  336.666656  360.123322   84164700
...                ...         ...         ...         ...        ...
2025-05-08  284.820007  289.799988  279.410004  279.630005   97539400
2025-05-09  298.260010  307.040009  290.000000  290.209991  132387800
2025-05-12  318.380005  322.209991  311.500000  321.989990  112826700
2025-05-13  334.070007  337.589996  316.799988  320.000000  136992600
2025-05-14  347.679993  350.000000  337.000000  342.500000  136997300

[844 


## Handle MultiIndex columns. (Removed complex indexing)/Flattening


In [102]:
for df in [treated_data, control_data, vix_data, tnx_data, sp500_data]:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)


## Debug: Printing the columns to confirm structure


In [103]:
print("Treated Data Columns (Tesla):", treated_data.columns)
print("Control Data Columns (Microsoft):", control_data.columns)
print("VIX Data Columns:", vix_data.columns)
print("10-Year Treasury Yield Columns:", tnx_data.columns)
print("S&P 500 Columns:", sp500_data.columns)

Treated Data Columns (Tesla): Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Control Data Columns (Microsoft): Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
VIX Data Columns: Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
10-Year Treasury Yield Columns: Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
S&P 500 Columns: Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')


## Check if 'Close' column exists in all DataFrames (Say no to API issues, hahaha)


In [104]:

for df, name in [(treated_data, 'treated_data (Tesla)'), (control_data, 'control_data (Microsoft)'),
                 (vix_data, 'VIX'), (tnx_data, '10-Year Treasury'), (sp500_data, 'S&P 500')]:
    if 'Close' not in df.columns:
        raise KeyError(f"'Close' column not found in {name}. Available columns: {df.columns}")

## Calculate daily returns for Tesla, Microsoft, and S&P 500

In [128]:
treated_data['returns'] = treated_data['Close'].pct_change() * 100 # Daily returns in percentage
control_data['returns'] = control_data['Close'].pct_change() * 100
sp500_data['returns'] = sp500_data['Close'].pct_change() * 100

print("Treated Data (Tesla) Daily Returns:\n", treated_data['returns'])

Treated Data (Tesla) Daily Returns:
 Date
2022-01-03         NaN
2022-01-04   -4.183270
2022-01-05   -5.347121
2022-01-06   -2.152337
2022-01-07   -3.544657
                ...   
2025-05-08    3.113462
2025-05-09    4.718770
2025-05-12    6.745790
2025-05-13    4.928074
2025-05-14    4.073992
Name: returns, Length: 844, dtype: float64


In [106]:
treated_data_monthly = treated_data.resample('M').last()
control_data_monthly = control_data.resample('M').last()
vix_data_monthly = vix_data.resample('M').last()
tnx_data_monthly = tnx_data.resample('M').last()
sp500_data_monthly = sp500_data.resample('M').last()

## Number of months


In [ ]:
n_months = len(treated_data_monthly)
print(f"Number of months in the dataset: {n_months}")

Number of months in the dataset: 41


## Define treatment period: Real Fed rate hike on July 26, 2023


In [108]:
treatment_date = pd.to_datetime('2023-07-26')
time = np.arange(n_months)

## Find the index of the nearest date to treatment_date


In [109]:
date_diffs = np.abs(treated_data_monthly.index - treatment_date)
nearest_idx = date_diffs.argmin()

## Create before_after and treated arrays


In [110]:
before_after = [0 if t < nearest_idx else 1 for t in time]
treated = [1 if t >= nearest_idx else 0 for t in time]

## Fetch P/E ratio (approximate using yfinance info)


In [ ]:
treated_info = yf.Ticker(treated_ticker).info
control_info = yf.Ticker(control_ticker).info
treated_pe = treated_info.get('trailingPE', 50)  # Fallback to 50 for Tesla
control_pe = control_info.get('trailingPE', 30)  # Fallback to 30 for Microsoft
print(f"Tesla PE Ratio: {treated_pe}")
print(f"Microsoft PE Ratio: {control_pe}")

Tesla PE Ratio: 198.74425
Microsoft PE Ratio: 35.227203


## Simulate P/E ratio variation over time (based on historical range)


In [112]:
pe_ratio_treated = np.random.normal(treated_pe, 10, n_months)  # Tesla: Higher volatility
pe_ratio_control = np.random.normal(control_pe, 5, n_months)   # Microsoft: More stable

## Simulate EBITDA (approximate realistic values for Tesla and Microsoft)


In [113]:
ebitda_treated = np.random.normal(1.5, 0.5, n_months)  # Tesla: ~$1.5B/month
ebitda_control = np.random.normal(10, 2, n_months)     # Microsoft: ~$10B/month
print(f"Tesla EBITDA: {ebitda_treated.mean()}")
print(f"Microsoft EBITDA: {ebitda_control.mean()}")

Tesla EBITDA: 1.4518811198893062
Microsoft EBITDA: 9.58660192676622


## Create DataFrame with additional confounders


In [114]:
data = pd.DataFrame({
    'time': np.concatenate([time, time]),
    'treated': np.concatenate([treated, [0] * n_months]),  # Control has treated=0
    'before_after': np.concatenate([before_after, before_after]),
    'returns': np.concatenate([treated_data_monthly['returns'].values, control_data_monthly['returns'].values]),
    'group': ['treated'] * n_months + ['control'] * n_months,
    'pe_ratio': np.concatenate([pe_ratio_treated, pe_ratio_control]),
    'ebitda': np.concatenate([ebitda_treated, ebitda_control]),
    'vix': np.concatenate([vix_data_monthly['Close'].values, vix_data_monthly['Close'].values]),
    'treasury_yield': np.concatenate([tnx_data_monthly['Close'].values, tnx_data_monthly['Close'].values]),
    'sp500_returns': np.concatenate([sp500_data_monthly['returns'].values, sp500_data_monthly['returns'].values])
})

# Step 2: Regression-based DiD with confounders and clustered standard errors


In [ ]:
data['treated_x_after'] = data['treated'] * data['before_after']
model = smf.ols(
    'returns ~ treated + before_after + treated_x_after + pe_ratio + ebitda + vix + treasury_yield + sp500_returns',
    data=data
).fit(cov_type='cluster', cov_kwds={'groups': data['time']})  # Clustered standard errors by time 
# (Standard errors --> measures uncertainty in models)
did_effect = model.params['treated_x_after'] # DiD effect coefficient
did_se = model.bse['treated_x_after'] # It shows uncertainty in the DiD effect
print(f"DiD Causal Effect: {did_effect:.2f}% (SE: {did_se:.3f}, p-value: {model.pvalues['treated_x_after']:.3f})")

DiD Causal Effect: -0.85% (SE: 0.426, p-value: 0.045)


# Step 3: CausalML T-Learner for heterogeneous treatment effects


In [121]:
X = data[['pe_ratio', 'ebitda', 'vix', 'treasury_yield', 'sp500_returns', 'before_after']]
treatment = data['treated']
y = data['returns']
t_learner = BaseTLearner(learner=RandomForestRegressor(n_estimators=100, random_state=369))
t_learner.fit(X, treatment, y)
cate = t_learner.predict(X)
data['cate'] = cate

# Step 4: Plot 1 - Stock returns over time


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='time', y='returns', hue='group', style='group', data=data)
plt.axvline(x=21, color='gray', linestyle='--', label='Fed Rate Hike (Jul 2023)')  # Set to 21st month
plt.xlabel('Months')
plt.ylabel('Stock Returns (%)')
plt.title(f'Stock Returns: Treated (TSLA) vs Control (MSFT) ({timestamp})')
plt.legend()
plt.grid(True)
plt.tight_layout(pad=2.0)
plt.savefig('stock_chart_advanced.png')
plt.close()

/opt/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


# Step 5: Plot 2 - Parallel trends assumption


In [118]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='time', y='returns', hue='group', style='group', data=data[data['time'] < nearest_idx])
plt.xlabel('Months (Pre-Treatment)')
plt.ylabel('Stock Returns (%)')
plt.title(f'Parallel Trends Check (Pre-Treatment) ({timestamp})')
plt.legend()
plt.grid(True)
plt.tight_layout(pad=2.0)
plt.savefig('parallel_trends.png')
plt.close()

/opt/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


# Step 6: Plot 3 - DiD summary with confidence intervals


In [119]:
summary = data.groupby(['group', 'before_after'])['returns'].mean().reset_index()
summary['before_after'] = summary['before_after'].map({0: 'Before', 1: 'After'})
plt.figure(figsize=(10, 6))
sns.lineplot(x='before_after', y='returns', hue='group', marker='o', data=summary)
plt.errorbar(
    x=[1], y=[did_effect + summary[(summary['group'] == 'control') & (summary['before_after'] == 'After')]['returns'].iloc[0]],
    yerr=1.96 * did_se, fmt='none', c='blue', capsize=5
)
plt.ylabel('Average Stock Returns (%)')
plt.title(f'DiD Effect of Fed Rate Hike (Jul 2023) ({timestamp})')
plt.grid(True)
plt.tight_layout(pad=2.0)
plt.savefig('did_plot_advanced.png')
plt.close()

/opt/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


# Step 7: Plot 4 - Treatment effect heterogeneity with trend line and improved scaling


In [129]:
plt.figure(figsize=(12, 6))
sns.scatterplot(
    x='pe_ratio', y='cate', hue='vix', size='treasury_yield', 
    sizes=(50, 500), palette='coolwarm',
    data=data[data['treated'] == 1]
)
sns.regplot(
    x='pe_ratio', y='cate', data=data[data['treated'] == 1], 
    scatter=False, color='black', lowess=True
)
plt.xlabel('P/E Ratio')
plt.ylabel('Estimated Treatment Effect (%)')
plt.title(f'Treatment Effect Heterogeneity by Confounders ({timestamp})')
plt.grid(True)
plt.tight_layout(pad=2.0)
plt.savefig('cate_heterogeneity.png')
plt.close()